# 인공 신경망 소개 with 케라스 

> 인공신경망은 뇌에 있는 생물학적 뉴런 네트워크에서 영감을 받은 머신러닝 모델 

- 딥러닝의 핵심

## 주요 내용 
- 인공신경망의 초장기 구조 소개 
- `다층 퍼셉트론` 구조 소개 
- `케라스`를 이용한 인공 신경망 구현 

# 1. 생물학적 뉴런에서 인공 뉴런까지 
## 생물학적 뉴런 
- `수상돌기` : 신호를 받아들임
- `축삭돌기` : 신경 전달 물질을 생성
- `시냅스` : 신경 전달 물질을 전달

## 뉴런을 사용한 논리 연산 
- `인공 뉴런` : 하나 이상의 이진 입력과 출력을 가짐
- 단순히 입력이 일정 개수 만큼 활성화 되었을 때 출력
- 이러한 구조를 통해 어떤 명제도 계산 가능

## 퍼셉트론 
### 정의
> 퍼셉트론이란 가장 간단한 인공 신경망 구조
- `TLU` 혹은 `LTU` 라고도 불림

### 원리 
- 입력과 출력이 이진값이 아닌 `숫자`이고, 각 입력 연결은 가중치와 연관됨 

#### 과정
1) 입력 선형 함수 계산 
  - 입력 선형 함수 = $z = w_1x_1 + w_2x_2 + ... w_nx_n + b = w^Tx+b $
2) `계단 함수` 계산 
  - 입력 선형 함수의 결과를 계단 함수 $h_w(x) = step(z)$에 적용 
  - `계단 함수` : 그래프를 그렸을 때, 계단처럼 층이 나눠진 함수 
  - 주로 `헤비사이드 계산함수` 혹은 `부호 함수`를 사용

#### 헤비사이드 계단 함수 
$$
\textrm{heaviside(z)} = \begin{cases}
   0 &\text{if } z<0 \\
   1 &\text{if } z \geq 0
\end{cases}
$$
  

#### 부호 함수 
$$
\textrm{sgn(z)} = \begin{cases}
   -1 &\text{if } z<0 \\
   0 &\text{if } z=0 \\
   +1 &\text{if } z> 0
\end{cases}
$$


### 퍼셉트론의 구조 
- `밀집층` : TLU가 모든 입력에 연결되어 있는 층
- `입력층` : 입력을 구성
- `출력층` : 최종 출력을 생성하는 층 

### 출력층의 계산 
$$
h_{W,b}(X) = \phi(XW + b)
$$

- $X$ : 입력 특성의 행렬
  - 행 : 샘플 
  - 열 : 특성 
- $W$ : 가중치 행렬 
  - 모든 연결 가중치 포함 
  - 행 : 입력 
  - 열 : 출력층의 뉴런 
- $b$ : 편향 벡터 
  - 모든 뉴런의 편향을 포함
- $\phi$ : 활성화 함수 
  - TLU의 경우 계단함수 

### 퍼셉트론의 훈련 
> 퍼셉트론은 `헤브의 규칙` 알고리즘을 이용하여 학습 
- 뉴런이 다른 뉴런을 활성화 시킬 때 이 두 뉴런의 연결이 강해진다고 제안 
- 즉, 뉴런이 `동시에 활성화` 될 때 마다 이들 사이의 `연결 가중치`가  증가하는 경향 

#### 원리 
- 퍼셉트론은 네트워크가 예측 할 때 만드는 오차를 반영하도록 조금 변형된 규칙을 사용하여 훈련
- `오차`가 `감소`되도록 연결을 강화 
  - 한 개의 샘플이 주입되면 각 샘플에 대한 예측이 만들어짐 
  - `잘못된 예측` -> 입력 가중치 강화 

#### 퍼셉트론의 가중치 업데이트 규칙
$$ 
w_{i,j}^{(\textrm{next step})} = w_{i,j} + \eta \cdot (y_j - \hat{y}_j) \cdot x_i
$$

|기호|의미|
|---|---|
|$w_{i,j}$ | i번째 입력 뉴런과 j 번째 출력 뉴런 사이 `연결 가중치`|
|$x_i$ | 현재 훈련 샘플의 i번째 뉴런의 `입력값`|
|$\hat{y}_j$ | 현재 훈련 샘플의 j번째 출력 뉴런의 `출력값`|
|$y_j$ | 현재 훈련 샘플의 j번째 뉴런의 `타깃값`|
|$\eta$ | 학습률|

#### 퍼셉트론 수렴 이론 
- 출력 뉴런의 결정 경계는 `선형`
  - 복잡한 패턴의 경우 학습 불가 
- 훈련 샘플이 선형적으로 구분 될 수 있다면 정답에 수렴할 수 있음 


### 사이킷런에서 퍼셉트론 구현하기 
- 클래스 : Perceptron
  - SGD로 구현 가능 
  - loss = "perceptron"
  - learning_rate="constant"
  - eat0=1
  - penalty=None

- 예시 : 붓꽃 데이터를 이용한 퍼셉트론 실습

In [4]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

### 붓꽃 데이터 로드 
iris = load_iris(as_frame=True)
X=iris.data[["petal length (cm)","petal width (cm)"]].values

### 종이 Iris-setosa 인 것만
y=(iris.target==0)

### 모델 학습
per_clf = Perceptron(random_state=42)
per_clf.fit(X,y)

###예측 시행 
X_new = [[2,0.5],[3,1]]
y_pred = per_clf.predict(X_new)
print("예측 확인 : ", y_pred)

예측 확인 :  [ True False]


## 다중 퍼셉트론(MLP)
> 퍼셉트론을 여러 개 쌓아올려 일부의 제약을 줄임 
- 퍼셉트론에서 사용 할 수 없었던 `XOR`을 `다중 퍼셉트론`에서는 해결 가능 

### 구성 
- 입력층 
  - 입력층과 가까운 층 = `하위층`
- 출력층 
  - 출력층과 가까운 층 = `상위층`
- `은닉층`
  - 은닉층을 여러 개 쌓아올린 신경망 = `심층신경망(DNN)`